# Kor LLM 모델 서빙

### 참조: 
- Model 정보
    - beomi/KoAlpaca-Polyglot-12.8B
        - This model is a fine-tuned version of EleutherAI/polyglot-ko-12.8b on a KoAlpaca Dataset v1.1b
        - https://huggingface.co/beomi/KoAlpaca-Polyglot-12.8B
    - EleutherAI/polyglot-ko-12.8b
        - Polyglot-Ko-12.8B was trained for 167 billion tokens over 301,000 steps on 256 A100 GPUs with the GPT-NeoX framework. It was trained as an autoregressive language model, using cross-entropy loss to maximize the likelihood of predicting the next token.
        - License: Apache 2.0
        - https://huggingface.co/EleutherAI/polyglot-ko-12.8b
        
- 블로그
    - https://aws.amazon.com/ko/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference/
- 코드
    - Boto3
        - https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/pytorch_deploy_large_GPT_model/GPT-J-6B-model-parallel-inference-DJL.ipynb
    - Python SDK
    - https://github.com/aws/amazon-sagemaker-examples/blob/main/inference/generativeai/deepspeed/GPT-J-6B_DJLServing_with_PySDK.ipynb

# 1. 기본 환경 설정

In [22]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('../common_code')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
!pip install -U langchain

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


# 2. SageMaker endpoint 의 추론 도커 이미지 인 DLC image URL 가져오기
- We get DLC image URL for djl-deepspeed 0.21.0 and set SageMaker settings

In [23]:
import sagemaker, boto3
from sagemaker import image_uris


role = sagemaker.get_execution_role()  # execution role for the endpoint
session = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = session._region_name
bucket = session.default_bucket()  # bucket to house artifacts

img_uri = image_uris.retrieve(framework="djl-deepspeed", region=region, version="0.21.0")
img_uri

'763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117'

# 3. Set configuration

## 테스트 모델 지정

In [24]:
#serve_model = 'KoAlpaca-12-8B'
#serve_model = 'Polyglot-Kor-5-8B'
serve_model = 'Kullm-polyglot-12-8b-v2'

In [25]:
model_artifact_name = f'{serve_model}.tar.gz'

instance_type = "ml.g5.12xlarge"
# instance_type = "ml.g5.48xlarge"

print("instance_type  :", instance_type)  


instance_type  : ml.g5.12xlarge


In [26]:
s3_location = f"s3://{bucket}/{serve_model}/"
s3_location

's3://sagemaker-us-east-1-419974056037/Kullm-polyglot-12-8b-v2/'

# 4. 모델 추론 코드 및 모델 설정 파일을 패키징
- `model.py` and `serving.properties`
- The code below creates the SageMaker model file (`model.tar.gz`) and upload it to S3. 

In [27]:
%%sh -s {serve_model} {model_artifact_name}
serve_model=$1
model_artifact_name=$2
echo $serve_model
echo $model_artifact_name

rm -rf $serve_model/.ipynb_checkpoints

tar -czvf $model_artifact_name $serve_model/



Kullm-polyglot-12-8b-v2
Kullm-polyglot-12-8b-v2.tar.gz
Kullm-polyglot-12-8b-v2/
Kullm-polyglot-12-8b-v2/serving.properties
Kullm-polyglot-12-8b-v2/model.py


## mode.tar.gz 를 S3 업로드

In [28]:
model_tar_url = sagemaker.s3.S3Uploader.upload(model_artifact_name, s3_location)

# 5. SageMaker endpoint 생성

- Now we create our [SageMaker model](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model). Make sure your execution role has access to your model artifacts and ECR image. Please check out our SageMaker Roles [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) for more details. 

In [29]:
from datetime import datetime

sm_client = boto3.client("sagemaker")

time_stamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
model_name = f"{serve_model}-" + time_stamp

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": img_uri, "ModelDataUrl": model_tar_url},
)

Now we create an endpoint configuration that SageMaker hosting services uses to deploy models. Note that we configured `ModelDataDownloadTimeoutInSeconds` and `ContainerStartupHealthCheckTimeoutInSeconds` to accommodate the large size of our model. 

In [30]:
initial_instance_count = 1
variant_name = "AllTraffic"
endpoint_config_name = f"{serve_model}-config-" + time_stamp

production_variants = [
    {
        "VariantName": variant_name,
        "ModelName": model_name,
        "InitialInstanceCount": initial_instance_count,
        "InstanceType": instance_type,
        "ModelDataDownloadTimeoutInSeconds": 300,
        "ContainerStartupHealthCheckTimeoutInSeconds": 300,
    }
]

endpoint_config = {
    "EndpointConfigName": endpoint_config_name,
    "ProductionVariants": production_variants,
}

ep_conf_res = sm_client.create_endpoint_config(**endpoint_config)

We are ready to create an endpoint using the model and the endpoint configuration created from above steps. 

In [31]:
endpoint_name = f"{serve_model}-" + time_stamp
ep_res = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

In [32]:
print("endpoint_name: ", endpoint_name)

endpoint_name:  Kullm-polyglot-12-8b-v2-2023-06-12-05-06-52


In [33]:
%%time 

from inference_lib import descirbe_endpoint
descirbe_endpoint(endpoint_name)            

Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating
Endpoint is  InService
CPU times: user 108 ms, sys: 24.6 ms, total: 132 ms
Wall time: 7min 1s


In [5]:
endpoint_name = "Polyglot-Kor-5-8B-2023-06-01-07-00-20"

# 6. 엔드포인트 추론 

In [34]:
from inference_lib import invoke_inference_DJ, Prompter

In [35]:
prompter = Prompter("kullm")

### options for generation
* **temperature**: Controls randomness in the model. Lower values will make the model more deterministic and higher values will make the model more random. Default value is 1.0.
* **max_new_tokens**: The maximum number of tokens to generate. Default value is 20, max value is 512.
* **repetition_penalty**: Controls the likelihood of repetition, defaults to null.
* **seed**: The seed to use for random generation, default is null.
* **stop**: A list of tokens to stop the generation. The generation will stop when one of the tokens is generated.
* **top_k**: The number of highest probability vocabulary tokens to keep for top-k-filtering. Default value is null, which disables top-k-filtering.
* **top_p**: The cumulative probability of parameter highest probability vocabulary tokens to keep for nucleus sampling, default to null
* **do_sample**: Whether or not to use sampling ; use greedy decoding otherwise. Default value is false.
* **best_of**: Generate best_of sequences and return the one if the highest token logprobs, default to null.
* **details**: Whether or not to return details about the generation. Default value is false.
* **return_full_text**: Whether or not to return the full text or only the generated part. Default value is false.
* **truncate**: Whether or not to truncate the input to the maximum length of the model. Default value is true.
* **typical_p**: The typical probability of a token. Default value is null.
* **watermark**: The watermark to use for the generation. Default value is false.

In [110]:
params = {
    "do_sample":False, 
    "max_new_tokens":256,
    "temperature":0.5,
    "top_k":3,
    "top_p":0.9,
    "return_full_text":False,
    "repetition_penalty":1.1,
    "presence_penalty":None,
    "eos_token_id":2,
}

## (1) 맥락 (Context) 없이 질문

In [72]:
q = "홈플러스 중계점은 몇시까지 장사해?"
c = ""#"홈플러스 영업시간은 오전 10시 부터 오후 12시까지 입니다."
prompt_wo_c = f"### 질문: {q}\n\n### 맥락: {c}\n\n### 답변:" if c else f"### 질문: {q}\n\n### 답변:" 
data = {
    "prompt": [prompt_wo_c,],
    "params": params
}
print("prompt_wo_c: \n", data)

prompt_wo_c: 
 {'prompt': ['### 질문: 홈플러스 중계점은 몇시까지 장사해?\n\n### 답변:'], 'params': {'do_sample': False, 'max_new_tokens': 128, 'temperature': 0.5, 'top_k': 3, 'top_p': 0.9, 'return_full_text': False, 'repetition_penalty': 1.1, 'presence_penalty': None, 'eos_token_id': 2}}


* **for kullm**

In [73]:
q = "홈플러스 중계점은 몇시까지 장사해?"
c = ""#"홈플러스 영업시간은 오전 10시 부터 오후 12시까지 입니다."
prompt_wo_c = prompter.generate_prompt(q, c)
data = {
    "prompt": [prompt_wo_c,],
    "params": params
}
print("prompt_wo_c: \n", data)

prompt_wo_c: 
 {'prompt': ['아래는 작업을 설명하는 명령어입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n홈플러스 중계점은 몇시까지 장사해?\n\n### 응답:\n'], 'params': {'do_sample': False, 'max_new_tokens': 128, 'temperature': 0.5, 'top_k': 3, 'top_p': 0.9, 'return_full_text': False, 'repetition_penalty': 1.1, 'presence_penalty': None, 'eos_token_id': 2}}


In [74]:
%%time 
invoke_inference_DJ(endpoint_name, data)

[
  [
    {
      "generated_text":"홈플러스 중계점의 영업시간은 매장마다 다를 수 있습니다. 홈플러스 중계점에 문의하여 영업 시간을 확인할 수 있습니다."
    }
  ]
]
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 1.82 s


'[\n  [\n    {\n      "generated_text":"홈플러스 중계점의 영업시간은 매장마다 다를 수 있습니다. 홈플러스 중계점에 문의하여 영업 시간을 확인할 수 있습니다."\n    }\n  ]\n]'

## (2) 맥락 (Context) 가지고 질문

In [75]:
q = "홈플러스 중계점은 몇시까지 장사해?"
c = "홈플러스 영업시간은 오전 10시 부터 오후 10시까지 입니다. 홈플러스 매장 찾기(영업시간 확인)는 이 주소를 이용하세요:  http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%B5%B5%B4%F6 "
prompt_w_c = f"### 질문: {q}\n\n### 맥락: {c}\n\n### 답변:" if c else f"### 질문: {q}\n\n### 답변:" 
data = {
    "prompt": [prompt_w_c,],
    "params": params
}
print("prompt_w_c:\n", prompt_w_c)

prompt_w_c:
 ### 질문: 홈플러스 중계점은 몇시까지 장사해?

### 맥락: 홈플러스 영업시간은 오전 10시 부터 오후 10시까지 입니다. 홈플러스 매장 찾기(영업시간 확인)는 이 주소를 이용하세요:  http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%B5%B5%B4%F6 

### 답변:


* **for kullm**

In [92]:
q = "홈플러스 중계점은 몇시까지 장사해?"
c = "홈플러스 영업시간은 오전 10시 부터 오후 10시까지 입니다. 홈플러스 매장 찾기(영업시간 확인)는 이 주소를 이용하세요:  http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%B5%B5%B4%F6 "
#prompt_w_c = f"### 질문: {q}\n\n### 맥락: {c}\n\n### 답변:" if c else f"### 질문: {q}\n\n### 답변:" 
prompt_w_c = prompter.generate_prompt(q, c)
data = {
    "prompt": [prompt_w_c,],
    "params": params
}
print("prompt_w_c:\n", prompt_w_c)

prompt_w_c:
 아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
홈플러스 중계점은 몇시까지 장사해?

### 입력:
홈플러스 영업시간은 오전 10시 부터 오후 10시까지 입니다. 홈플러스 매장 찾기(영업시간 확인)는 이 주소를 이용하세요:  http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%B5%B5%B4%F6 

### 응답:



In [93]:
%%time 
invoke_inference_DJ(endpoint_name, data)

[
  [
    {
      "generated_text":"홈플러스 중계점의 영업시간은 오전 10시부터 오후 10시까지입니다."
    }
  ]
]
CPU times: user 10.2 ms, sys: 2.29 ms, total: 12.5 ms
Wall time: 1.17 s


'[\n  [\n    {\n      "generated_text":"홈플러스 중계점의 영업시간은 오전 10시부터 오후 10시까지입니다."\n    }\n  ]\n]'

## (3) 맥락 (Context) 에 없는 질문

In [102]:
q = "오미크론 변종이 등장한 후 경제가 위축되었나요?"
c = """2020년과 2021년에는 약 4조 4천억 달러, 즉 2021년 국내 총생산 (GDP) 의 18% 에 달하는 막대한 QE와 2021년 GDP의 약 5조 달러, 즉 21% 에 달하는 막대한 재정 부양책 (지금까지도 인플레이션일 것입니다) 이 시장을 안정시켰고 기업들은 막대한 자본을 조달할 수 있었습니다.또한 이러한 자본 투입을 통해 많은 중소기업을 구했고 소비자에게는 2조 5천억 달러 이상을, 주 및 지방 금고에 거의 1조 달러를 투자했습니다.이러한 조치로 인해 실업률이 20개월 만에 15% 에서 4% 미만으로 급격히 감소했습니다. 그 규모와 속도는 전례 없는 일이었습니다.또한 델타와 오미크론 변종의 출현과 글로벌 공급망 부족에도 불구하고 2021년 경제는 7% 성장했습니다. 이는 주로 소비자 지출의 급격한 증가와 해당 지출의 서비스에서 상품으로의 전환에 힘입은 것입니다.다행히 이 2년 동안 COVID-19 백신도 빠르게 개발되어 배포되었습니다.
오늘날 경제에서 소비자들은 (평균적으로) 우수한 재무 상태를 유지하고 있으며, 레버리지는 사상 최저 수준이며, 주택 가격 상승이 있었음에도 불구하고 우수한 모기지 인수, 임금 인상으로 인한 많은 일자리 및 2조 달러 이상의 초과 저축을 이루고 있습니다. 이는 주로 정부 경기 부양책으로 인한 것입니다.대부분의 소비자와 기업 (및 주) 은 2020년과 2021년에 창출된 자금으로 여전히 넘쳐나고 있으며, 지난 몇 개월 동안의 소비자 지출은 COVID-19 이전 수준보다 12% 높습니다.(그러나 우리는 저소득 가구의 계좌 잔고가 처음에는 규모가 작을수록 더 빠르게 감소하고 있으며 해당 가구의 소득이 인플레이션 상승에 보조를 맞추지 못하고 있다는 점을 인식해야 합니다.)
오늘날의 경제 상황은 2008년 금융 위기와 완전히 달라졌습니다. 은행과 투자 은행에서부터 섀도우 뱅크, 헤지 펀드, 사모펀드, 패니 매 (Fannie Mae) 및 기타 여러 기업에 이르기까지 금융 시스템 전체가 그랬듯이 소비자가 과도하게 레버리지를 받았을 때였습니다.또한 주택 담보 대출 제도의 부실 인수 및 레버리지로 인한 주택 가격 상승은 과도한 투기로 이어져 거의 모든 사람이 이를 놓쳤고 결국 거의 1조 달러에 가까운 실제 손실을 초래했습니다."""
prompt_w_c = prompter.generate_prompt(q, c)
data = {
    "prompt": [prompt_w_c,],
    "params": params
}
print("prompt_w_c:\n", prompt_w_c)

prompt_w_c:
 아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
오미크론 변종이 등장한 후 경제가 위축되었나요?

### 입력:
2020년과 2021년에는 약 4조 4천억 달러, 즉 2021년 국내 총생산 (GDP) 의 18% 에 달하는 막대한 QE와 2021년 GDP의 약 5조 달러, 즉 21% 에 달하는 막대한 재정 부양책 (지금까지도 인플레이션일 것입니다) 이 시장을 안정시켰고 기업들은 막대한 자본을 조달할 수 있었습니다.또한 이러한 자본 투입을 통해 많은 중소기업을 구했고 소비자에게는 2조 5천억 달러 이상을, 주 및 지방 금고에 거의 1조 달러를 투자했습니다.이러한 조치로 인해 실업률이 20개월 만에 15% 에서 4% 미만으로 급격히 감소했습니다. 그 규모와 속도는 전례 없는 일이었습니다.또한 델타와 오미크론 변종의 출현과 글로벌 공급망 부족에도 불구하고 2021년 경제는 7% 성장했습니다. 이는 주로 소비자 지출의 급격한 증가와 해당 지출의 서비스에서 상품으로의 전환에 힘입은 것입니다.다행히 이 2년 동안 COVID-19 백신도 빠르게 개발되어 배포되었습니다.
오늘날 경제에서 소비자들은 (평균적으로) 우수한 재무 상태를 유지하고 있으며, 레버리지는 사상 최저 수준이며, 주택 가격 상승이 있었음에도 불구하고 우수한 모기지 인수, 임금 인상으로 인한 많은 일자리 및 2조 달러 이상의 초과 저축을 이루고 있습니다. 이는 주로 정부 경기 부양책으로 인한 것입니다.대부분의 소비자와 기업 (및 주) 은 2020년과 2021년에 창출된 자금으로 여전히 넘쳐나고 있으며, 지난 몇 개월 동안의 소비자 지출은 COVID-19 이전 수준보다 12% 높습니다.(그러나 우리는 저소득 가구의 계좌 잔고가 처음에는 규모가 작을수록 더 빠르게 감소하고 있으며 해당 가구의 소득이 인플레이션 상승에 보조를 맞추지 못하고 있다는 점을 인식해야 합니다.)
오늘날의 경제 상황은 

In [103]:
%%time 
invoke_inference_DJ(endpoint_name, data)

[
  [
    {
      "generated_text":"경제가 위축되지 않았습니다. 코로나19 팬데믹 기간 동안 대부분의 소비자들이 우수한 재무 상태를 유지했으며, 레버리지는 사상 최저치를 기록했습니다. 또한, 소비자들은 코로나19 백신의 빠른 개발 덕분에 2021년 경제가 7% 성장할 수 있었습니다. 그러나 저소득 가구의 계좌 잔고는 처음에는 규모가 작을수록 더 빠르게 감소하고 있으며, 해당 가구의 소득이 인플레이션 상승에 보조를 맞추지 못하고 있습니다."
    }
  ]
]
CPU times: user 12.1 ms, sys: 667 µs, total: 12.8 ms
Wall time: 6.22 s


'[\n  [\n    {\n      "generated_text":"경제가 위축되지 않았습니다. 코로나19 팬데믹 기간 동안 대부분의 소비자들이 우수한 재무 상태를 유지했으며, 레버리지는 사상 최저치를 기록했습니다. 또한, 소비자들은 코로나19 백신의 빠른 개발 덕분에 2021년 경제가 7% 성장할 수 있었습니다. 그러나 저소득 가구의 계좌 잔고는 처음에는 규모가 작을수록 더 빠르게 감소하고 있으며, 해당 가구의 소득이 인플레이션 상승에 보조를 맞추지 못하고 있습니다."\n    }\n  ]\n]'

In [106]:
q = "COVID-19가 결국 경제에 도움이 되었나요?"
c = """2020년과 2021년에는 약 4조 4천억 달러, 즉 2021년 국내 총생산 (GDP) 의 18% 에 달하는 막대한 QE와 2021년 GDP의 약 5조 달러, 즉 21% 에 달하는 막대한 재정 부양책 (지금까지도 인플레이션일 것입니다) 이 시장을 안정시켰고 기업들은 막대한 자본을 조달할 수 있었습니다.또한 이러한 자본 투입을 통해 많은 중소기업을 구했고 소비자에게는 2조 5천억 달러 이상을, 주 및 지방 금고에 거의 1조 달러를 투자했습니다.이러한 조치로 인해 실업률이 20개월 만에 15% 에서 4% 미만으로 급격히 감소했습니다. 그 규모와 속도는 전례 없는 일이었습니다.또한 델타와 오미크론 변종의 출현과 글로벌 공급망 부족에도 불구하고 2021년 경제는 7% 성장했습니다. 이는 주로 소비자 지출의 급격한 증가와 해당 지출의 서비스에서 상품으로의 전환에 힘입은 것입니다.다행히 이 2년 동안 COVID-19 백신도 빠르게 개발되어 배포되었습니다.
오늘날 경제에서 소비자들은 (평균적으로) 우수한 재무 상태를 유지하고 있으며, 레버리지는 사상 최저 수준이며, 주택 가격 상승이 있었음에도 불구하고 우수한 모기지 인수, 임금 인상으로 인한 많은 일자리 및 2조 달러 이상의 초과 저축을 이루고 있습니다. 이는 주로 정부 경기 부양책으로 인한 것입니다.대부분의 소비자와 기업 (및 주) 은 2020년과 2021년에 창출된 자금으로 여전히 넘쳐나고 있으며, 지난 몇 개월 동안의 소비자 지출은 COVID-19 이전 수준보다 12% 높습니다.(그러나 우리는 저소득 가구의 계좌 잔고가 처음에는 규모가 작을수록 더 빠르게 감소하고 있으며 해당 가구의 소득이 인플레이션 상승에 보조를 맞추지 못하고 있다는 점을 인식해야 합니다.)
오늘날의 경제 상황은 2008년 금융 위기와 완전히 달라졌습니다. 은행과 투자 은행에서부터 섀도우 뱅크, 헤지 펀드, 사모펀드, 패니 매 (Fannie Mae) 및 기타 여러 기업에 이르기까지 금융 시스템 전체가 그랬듯이 소비자가 과도하게 레버리지를 받았을 때였습니다.또한 주택 담보 대출 제도의 부실 인수 및 레버리지로 인한 주택 가격 상승은 과도한 투기로 이어져 거의 모든 사람이 이를 놓쳤고 결국 거의 1조 달러에 가까운 실제 손실을 초래했습니다."""
prompt_w_c = prompter.generate_prompt(q, c)
data = {
    "prompt": [prompt_w_c,],
    "params": params
}
print("prompt_w_c:\n", prompt_w_c)

prompt_w_c:
 아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
COVID-19가 결국 경제에 도움이 되었나요?

### 입력:
2020년과 2021년에는 약 4조 4천억 달러, 즉 2021년 국내 총생산 (GDP) 의 18% 에 달하는 막대한 QE와 2021년 GDP의 약 5조 달러, 즉 21% 에 달하는 막대한 재정 부양책 (지금까지도 인플레이션일 것입니다) 이 시장을 안정시켰고 기업들은 막대한 자본을 조달할 수 있었습니다.또한 이러한 자본 투입을 통해 많은 중소기업을 구했고 소비자에게는 2조 5천억 달러 이상을, 주 및 지방 금고에 거의 1조 달러를 투자했습니다.이러한 조치로 인해 실업률이 20개월 만에 15% 에서 4% 미만으로 급격히 감소했습니다. 그 규모와 속도는 전례 없는 일이었습니다.또한 델타와 오미크론 변종의 출현과 글로벌 공급망 부족에도 불구하고 2021년 경제는 7% 성장했습니다. 이는 주로 소비자 지출의 급격한 증가와 해당 지출의 서비스에서 상품으로의 전환에 힘입은 것입니다.다행히 이 2년 동안 COVID-19 백신도 빠르게 개발되어 배포되었습니다.
오늘날 경제에서 소비자들은 (평균적으로) 우수한 재무 상태를 유지하고 있으며, 레버리지는 사상 최저 수준이며, 주택 가격 상승이 있었음에도 불구하고 우수한 모기지 인수, 임금 인상으로 인한 많은 일자리 및 2조 달러 이상의 초과 저축을 이루고 있습니다. 이는 주로 정부 경기 부양책으로 인한 것입니다.대부분의 소비자와 기업 (및 주) 은 2020년과 2021년에 창출된 자금으로 여전히 넘쳐나고 있으며, 지난 몇 개월 동안의 소비자 지출은 COVID-19 이전 수준보다 12% 높습니다.(그러나 우리는 저소득 가구의 계좌 잔고가 처음에는 규모가 작을수록 더 빠르게 감소하고 있으며 해당 가구의 소득이 인플레이션 상승에 보조를 맞추지 못하고 있다는 점을 인식해야 합니다.)
오늘날의 경제 상황은 

In [107]:
%%time 
invoke_inference_DJ(endpoint_name, data)

[
  [
    {
      "generated_text":"COVID-19 팬데믹은 경제 회복에 큰 영향을 미쳤습니다. 팬데믹 기간 동안 대부분의 소비자들이 우수한 재무 상태를 유지했으며, 이는 소비자 지출의 급격한 증가로 이어졌습니다. 또한, 기업들은 막대한 자본을 조달하여 고용을 늘리고 소비자에게 막대한 금액을 투자할 수 있었습니다. 그러나 이러한 조치들은 인플레이션을 유발할 수도 있으며, 저소득 가구의 계좌 잔고는 처음에는 규모가 작을수록 더 빠르게 감소한다는 문제점이 있습니다. 또한, 금융 시스템 전반에 걸쳐 소비자들이 과도한 레버리지를 받았던 2008년 금융 위기"
    }
  ]
]
CPU times: user 12.5 ms, sys: 0 ns, total: 12.5 ms
Wall time: 7.69 s


'[\n  [\n    {\n      "generated_text":"COVID-19 팬데믹은 경제 회복에 큰 영향을 미쳤습니다. 팬데믹 기간 동안 대부분의 소비자들이 우수한 재무 상태를 유지했으며, 이는 소비자 지출의 급격한 증가로 이어졌습니다. 또한, 기업들은 막대한 자본을 조달하여 고용을 늘리고 소비자에게 막대한 금액을 투자할 수 있었습니다. 그러나 이러한 조치들은 인플레이션을 유발할 수도 있으며, 저소득 가구의 계좌 잔고는 처음에는 규모가 작을수록 더 빠르게 감소한다는 문제점이 있습니다. 또한, 금융 시스템 전반에 걸쳐 소비자들이 과도한 레버리지를 받았던 2008년 금융 위기"\n    }\n  ]\n]'

In [111]:
q = "COVID-19가 2008년의 금융 위기에 어떻게 영향을 주었나요?"
c = """2020년과 2021년에는 약 4조 4천억 달러, 즉 2021년 국내 총생산 (GDP) 의 18% 에 달하는 막대한 QE와 2021년 GDP의 약 5조 달러, 즉 21% 에 달하는 막대한 재정 부양책 (지금까지도 인플레이션일 것입니다) 이 시장을 안정시켰고 기업들은 막대한 자본을 조달할 수 있었습니다.또한 이러한 자본 투입을 통해 많은 중소기업을 구했고 소비자에게는 2조 5천억 달러 이상을, 주 및 지방 금고에 거의 1조 달러를 투자했습니다.이러한 조치로 인해 실업률이 20개월 만에 15% 에서 4% 미만으로 급격히 감소했습니다. 그 규모와 속도는 전례 없는 일이었습니다.또한 델타와 오미크론 변종의 출현과 글로벌 공급망 부족에도 불구하고 2021년 경제는 7% 성장했습니다. 이는 주로 소비자 지출의 급격한 증가와 해당 지출의 서비스에서 상품으로의 전환에 힘입은 것입니다.다행히 이 2년 동안 COVID-19 백신도 빠르게 개발되어 배포되었습니다.
오늘날 경제에서 소비자들은 (평균적으로) 우수한 재무 상태를 유지하고 있으며, 레버리지는 사상 최저 수준이며, 주택 가격 상승이 있었음에도 불구하고 우수한 모기지 인수, 임금 인상으로 인한 많은 일자리 및 2조 달러 이상의 초과 저축을 이루고 있습니다. 이는 주로 정부 경기 부양책으로 인한 것입니다.대부분의 소비자와 기업 (및 주) 은 2020년과 2021년에 창출된 자금으로 여전히 넘쳐나고 있으며, 지난 몇 개월 동안의 소비자 지출은 COVID-19 이전 수준보다 12% 높습니다.(그러나 우리는 저소득 가구의 계좌 잔고가 처음에는 규모가 작을수록 더 빠르게 감소하고 있으며 해당 가구의 소득이 인플레이션 상승에 보조를 맞추지 못하고 있다는 점을 인식해야 합니다.)
오늘날의 경제 상황은 2008년 금융 위기와 완전히 달라졌습니다. 은행과 투자 은행에서부터 섀도우 뱅크, 헤지 펀드, 사모펀드, 패니 매 (Fannie Mae) 및 기타 여러 기업에 이르기까지 금융 시스템 전체가 그랬듯이 소비자가 과도하게 레버리지를 받았을 때였습니다.또한 주택 담보 대출 제도의 부실 인수 및 레버리지로 인한 주택 가격 상승은 과도한 투기로 이어져 거의 모든 사람이 이를 놓쳤고 결국 거의 1조 달러에 가까운 실제 손실을 초래했습니다."""
prompt_w_c = prompter.generate_prompt(q, c)
data = {
    "prompt": [prompt_w_c,],
    "params": params
}
print("prompt_w_c:\n", prompt_w_c)

prompt_w_c:
 아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
COVID-19가 2008년의 금융 위기에 어떻게 영향을 주었나요?

### 입력:
2020년과 2021년에는 약 4조 4천억 달러, 즉 2021년 국내 총생산 (GDP) 의 18% 에 달하는 막대한 QE와 2021년 GDP의 약 5조 달러, 즉 21% 에 달하는 막대한 재정 부양책 (지금까지도 인플레이션일 것입니다) 이 시장을 안정시켰고 기업들은 막대한 자본을 조달할 수 있었습니다.또한 이러한 자본 투입을 통해 많은 중소기업을 구했고 소비자에게는 2조 5천억 달러 이상을, 주 및 지방 금고에 거의 1조 달러를 투자했습니다.이러한 조치로 인해 실업률이 20개월 만에 15% 에서 4% 미만으로 급격히 감소했습니다. 그 규모와 속도는 전례 없는 일이었습니다.또한 델타와 오미크론 변종의 출현과 글로벌 공급망 부족에도 불구하고 2021년 경제는 7% 성장했습니다. 이는 주로 소비자 지출의 급격한 증가와 해당 지출의 서비스에서 상품으로의 전환에 힘입은 것입니다.다행히 이 2년 동안 COVID-19 백신도 빠르게 개발되어 배포되었습니다.
오늘날 경제에서 소비자들은 (평균적으로) 우수한 재무 상태를 유지하고 있으며, 레버리지는 사상 최저 수준이며, 주택 가격 상승이 있었음에도 불구하고 우수한 모기지 인수, 임금 인상으로 인한 많은 일자리 및 2조 달러 이상의 초과 저축을 이루고 있습니다. 이는 주로 정부 경기 부양책으로 인한 것입니다.대부분의 소비자와 기업 (및 주) 은 2020년과 2021년에 창출된 자금으로 여전히 넘쳐나고 있으며, 지난 몇 개월 동안의 소비자 지출은 COVID-19 이전 수준보다 12% 높습니다.(그러나 우리는 저소득 가구의 계좌 잔고가 처음에는 규모가 작을수록 더 빠르게 감소하고 있으며 해당 가구의 소득이 인플레이션 상승에 보조를 맞추지 못하고 있다는 점을 인식해야 합니다.)
오

In [112]:
%%time 
invoke_inference_DJ(endpoint_name, data)

[
  [
    {
      "generated_text":"2008년의 금융 위기는 소비자들이 과도한 레버리지를 받고, 주택 가격이 급등하여 과도한 투기로 이어진 결과였습니다. 이로 인해 많은 소비자들이 주택을 구입하거나 유지하기에 충분한 돈을 모으지 못한 채 대출을 받아야 했습니다. 또한, 금융 기관들은 주택 담보 대출을 무분별하게 확대하여 주택 가격이 더욱 상승하도록 만들었습니다. 그러나 코로나19 팬데믹 이후, 소비자들은 안정적인 재무 상태를 유지했으며, 레버리지는 사상 최저치를 기록했습니다. 또한, 소비자들은 임금 인상으로 인해 많은 일자리를 얻었고, 저축액도 크게 증가했습니다. 이러한 변화로 인해 경제는 빠르게 회복되었지만, 저소득 가구의 계좌 잔고는 처음에는 규모가 작을수록 더 빠르게 감소하는 경향이 있었습니다."
    }
  ]
]
CPU times: user 12.9 ms, sys: 127 µs, total: 13 ms
Wall time: 10.6 s


'[\n  [\n    {\n      "generated_text":"2008년의 금융 위기는 소비자들이 과도한 레버리지를 받고, 주택 가격이 급등하여 과도한 투기로 이어진 결과였습니다. 이로 인해 많은 소비자들이 주택을 구입하거나 유지하기에 충분한 돈을 모으지 못한 채 대출을 받아야 했습니다. 또한, 금융 기관들은 주택 담보 대출을 무분별하게 확대하여 주택 가격이 더욱 상승하도록 만들었습니다. 그러나 코로나19 팬데믹 이후, 소비자들은 안정적인 재무 상태를 유지했으며, 레버리지는 사상 최저치를 기록했습니다. 또한, 소비자들은 임금 인상으로 인해 많은 일자리를 얻었고, 저축액도 크게 증가했습니다. 이러한 변화로 인해 경제는 빠르게 회복되었지만, 저소득 가구의 계좌 잔고는 처음에는 규모가 작을수록 더 빠르게 감소하는 경향이 있었습니다."\n    }\n  ]\n]'

# 7. [중요] 클린업 엔트포인트 

In [55]:
# sm_client.delete_endpoint(EndpointName=endpoint_name)
# sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
# sm_client.delete_model(ModelName=model_name)

In [22]:
from inference_lib import Prompter
prompter = Prompter("kullm")

In [23]:
instruction = "홈플러스 중계점은 몇시까지 장사해?"
input_text = "홈플러스 영업시간은 오전 10시 부터 오후 10시까지 입니다. 홈플러스 매장 찾기(영업시간 확인)는 이 주소를 이용하세요:  http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%B5%B5%B4%F6 "

prompt = prompter.generate_prompt(instruction, input_text)

In [24]:
prompt

'아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n홈플러스 중계점은 몇시까지 장사해?\n\n### 입력:\n홈플러스 영업시간은 오전 10시 부터 오후 10시까지 입니다. 홈플러스 매장 찾기(영업시간 확인)는 이 주소를 이용하세요:  http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%B5%B5%B4%F6 \n\n### 응답:\n'